In [16]:
!pip -q install langchain==0.0.179 pydantic tiktoken chromadb pypdf Xformers InstructorEmbedding sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
!pip show langchain

Name: langchain
Version: 0.0.179
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [28]:
!wget -q https://www.dropbox.com/s/zoj9rnm7oyeaivb/new_papers.zip
!unzip -q new_papers.zip -d new_papers

replace new_papers/new_papers/toolformer.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
import re
import sys
import json
import torch
import logging
from random import choice
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

In [3]:
logger = logging.getLogger('api')
logger.setLevel(logging.INFO)

logHandler = logging.StreamHandler(sys.stdout)
logger.addHandler(logHandler)

In [4]:
def setup_model(model_name: str, cache_dir: str = None):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        cache_dir=cache_dir
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map='auto',
        low_cpu_mem_usage=True,
        cache_dir=cache_dir,
    )
    return tokenizer, model

In [6]:
model_name='TheBloke/wizardLM-7B-HF'
cache_dir='/home/ec2-user/SageMaker/.cache'
tokenizer, model = setup_model(model_name, cache_dir)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so'), PosixPath('/home/ec2-user/anaconda3/envs/pytorch_p39/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
!nvidia-smi

Thu May 25 13:32:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   25C    P0    61W / 300W |   7931MiB / 23028MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,   
)

In [10]:
llm = HuggingFacePipeline(pipeline=pipe)
print(llm('What is the capital of England?'))


London is the capital city of England.


## Setup Langchain

In [26]:
import textwrap
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [18]:
loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [19]:
len(documents)

142

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

## Embbeding

In [21]:
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl", 
    model_kwargs={"device": "cuda"}
)

load INSTRUCTOR_Transformer
max_seq_length  512


## Create DB

In [22]:
embedding = instructor_embeddings
vectordb = Chroma.from_documents(
    documents=texts, 
    embedding=embedding,
    persist_directory='db',
)

## Set chain

In [23]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [27]:
def wrap_text_preserve_newlines(text, width=110):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## Run

In [28]:
%%time

query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 FlashAttention is a new attention algorithm proposed by the authors that reduces the number of memory
reads/writes between GPU high bandwidth memory (HBM) and GPU on-chip SRAM. It achieves this by splitting the
input into blocks and making several passes over them, and by storing the softmax normalization factor from
the forward pass to quickly recompute attention on-chip in the backward pass. FlashAttention also extends to
block-sparse attention, yielding an approximate attention algorithm that is faster than any existing
approximate attention method.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [29]:
%%time

query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 IO-aware refers to a method or approach that takes into account the input/output (I/O) performance when
designing or optimizing a system. In the context of deep learning, it means considering the impact of the
computational load on the input/output devices (such as CPUs, GPUs, or other accelerators) during the training
process.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
CPU times: user 9.63 s, sys: 0 ns, total: 9.63 s
Wall time: 9.62 s


In [30]:
%%time

query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that has been trained to use external tools such as search engines,
calculators, and translation systems via simple API calls. It was introduced in a recent research paper titled
"Learning to Use Tools with Language Models" by Timo Schick, Jane Dwivedi-Yu, Roberto Dessìy, Roberta
Raileanu, Maria Lomeli, Luke Zettlemoyer, Nicola Cancedda, Thomas Scialom, Meta AI ResearchyUniversitat Pompeu
Fabra.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
CPU times: user 15.1 s, sys: 0 ns, total: 15.1 s
Wall time: 15.1 s


In [31]:
%%time

query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Toolformer can be used with any external tool that has a simple API and provides relevant information for the
given task. For example, it could be used with search engines like Google or Bing, calculators like Wolfram
Alpha or Mathway, or translation systems like Google Translate or Microsoft Translator.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
CPU times: user 8.76 s, sys: 3.68 ms, total: 8.76 s
Wall time: 8.75 s


In [32]:
%%time 

query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

There are different types of retrieval augmentations for LLMs, including dense and sparse retrievers. Dense
retrievers work with dense queries and dense document representations, while sparse retrievers use sparse bag-
of-words representations of the documents and queries. Both approaches have their advantages and
disadvantages, and the choice depends on the specific task and dataset. Additionally, grounding the
predictions through tools such as calculators can increase the truthfulness of the generated responses.
Estimating and reducing uncertainty is another direction to explore, as it can help LMMs learn what they know
and what they don't. Finally, allowing LMMs to leverage external tools can also improve their performance,
especially if the missing information is crucial for the task. Overall, the best retrieval augmentations
depend on the specific requirements of the task and should be evaluated based on their effectiveness and
efficiency.


Sources:
new_papers/new_papers/Augmenti

In [33]:
%%time

query = "What are the differences between REALM and RAG?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

REALM (Guu et al., 2020) and RAG (Lewis et al., 2020) are both methods that use retrieval-augmented language
models to improve the performance of question answering systems. However, there are some key differences
between them. REALM


Sources:
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_papers/new_papers/ReACT.pdf
CPU times: user 8.51 s, sys: 0 ns, total: 8.51 s
Wall time: 8.5 s


In [34]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f2e71b13eb0>)

In [35]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:
